In [2]:
%pip install -Uq upgini catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.4/434.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.9/913.9 kB 50.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━

In [3]:
from os.path import exists
import pandas as pd

df_path= "train.csv.zip" if exists ("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df= pd.read_csv(df_path)
df=df.sample(n=19_000,random_state=0)
df["store"]= df["store"].astype(str)
df["item"]=df["item"].astype(str)

df["date"]= pd.to_datetime(df["date"])

df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop= True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [6]:
train= df[df["date"] < "2017-01-01"]
test= df[df["date"] >= "2017-01-01"]

In [17]:
train_features= train.drop(columns=["sales"])
train_target= train["sales"]
test_features= test.drop(columns=["sales"])
test_target = test["sales"]

Enrich Features

In [18]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher= FeaturesEnricher(
    search_keys= {
        "date": SearchKey.DATE,
    },
    Cv= CVType.time_series
)
enricher.fit (train_features,
             train_target,
             eval_set=[(test_features, test_target)])

[============================================================] 100% Finished


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history

Detected task type: ModelTaskType.REGRESSION




Column name,Status,Errors
target,All valid,-
date,All valid,-



Running search request, search_id=43b9cd52-cc29-43a1-89b1-ca823671e774
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


67 relevant feature(s) found with the search keys: ['date']


f_autofe_div_89c56a5f,0.0182,100.0000,"1.7292, 1.4026, -0.8598",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_cf5dd7be,0.0104,100.0000,"0.787, -0.8765, 0.5585",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_08e89348,0.0083,100.0000,"47.9275, -23.0171, -48.0932",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_events_date_year_cos1_9014a856,0.0079,100.0000,"-0.5913, 0.4674, -0.6839",Upgini,Calendar data,Daily
f_autofe_mul_478be4f3,0.0071,100.0000,"0.9509, 0.2682, -0.0318",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_692a5456,0.0059,100.0000,"47.8185, 87.1963, 45.8129",Upgini,AutoFE: features from Markets data,Daily
f_autofe_mul_5bd50798,0.0058,100.0000,"0.675, -1.2264, -1.0066",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_b6456080,0.0050,100.0000,"0.681, 1.2619, 0.8614",Upgini,AutoFE: features from Markets data,Daily
f_autofe_mul_0870be07,0.0043,100.0000,"20.8882, -7.7126, 2.0089",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_73b33182,0.0043,100.0000,"-0.1515, -0.0424, 0.4182",Upgini,AutoFE: features from Calendar data,Daily
f_autofe_div_3e22df83,0.0042,100.0000,"0.0149, 0.0078, -0.0099",Upgini,"AutoFE: features from Calendar data,Markets data",Daily


Upgini,"AutoFE: features from Calendar data,Markets data",0.1146,37
Upgini,AutoFE: features from Markets data,0.0290,16
Upgini,AutoFE: features from Calendar data,0.0166,10
Upgini,Calendar data,0.0079,1
Upgini,AutoFE: feature from Markets data,0.0044,2
Upgini,Markets data,0.0011,1


"Calendar data,Markets data",f_autofe_div_89c56a5f,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,/
"Calendar data,Markets data",f_autofe_div_cf5dd7be,f_events_date_week_sin1_847b5db1,f_financial_date_vix_7d_to_1y_634c77eb,/
"Calendar data,Markets data",f_autofe_mul_08e89348,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,*
"Calendar data,Markets data",f_autofe_mul_478be4f3,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,*
Markets data,f_autofe_mul_692a5456,f_financial_date_crude_oil_1f195998,f_financial_date_silver_7d_to_7d_1y_shift_55fa8001,*
"Calendar data,Markets data",f_autofe_mul_5bd50798,f_events_date_week_cos1_f6a8c1fc,f_financial_date_nasdaq_7d_to_7d_1y_shift_4a4dd6ce,*
Markets data,f_autofe_div_b6456080,f_financial_date_gold_7d_to_1y_ae310379,f_financial_date_stoxx_7d_to_7d_1y_shift_abef6e91,/
"Calendar data,Markets data",f_autofe_mul_0870be07,f_events_date_year_cos2_7acbaf1b,f_financial_date_vix_6c8e6194,*
Calendar data,f_autofe_mul_73b33182,f_events_date_year_cos4_f5e6853f,f_events_date_year_sin1_3c44bc64,*
"Calendar data,Markets data",f_autofe_div_3e22df83,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,/
"Calendar data,Markets data",f_autofe_mul_00f83c53,f_events_date_week_cos2_b0a07cfc,f_financial_date_gold_7d_to_1y_ae310379,*



Examples of outliers with maximum value of target:
84    205
47    196
38    187
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...

which makes metrics between the train and eval_set incomparable.


Train,15213,50.3977,319.7111,226.8107,92.9005
Eval 1,3787,59.2424,443.8766,341.2927,102.5839


In [26]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric
model = CatBoostRegressor(verbose=False, allow_writing_files=False,random_state=0)

enricher.calculate_metrics(
    train_features,train_target,
    eval_set=[(test_features,test_target)],
    estimator= model,
    scoring= "mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
-
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,15213,50.3977,0.314875,0.198207,0.116668
1,Eval 1,3787,59.2424,0.281885,0.209974,0.071911


In [27]:
enriched_train_features= enricher.transform(train_features,keep_input=True)
enriched_test_features= enricher.transform(test_features,keep_input=True)

[============================================================] 100% Finished

You are trying to launch enrichment for 15213 rows, which will exceed the rest limit 10000.


Button(button_style='danger', description='Request a quote', style=ButtonStyle())

[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 0 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-



Running search request, search_id=3d831c9e-c69e-49fc-94da-999bcbd37640
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


In [28]:
model.fit(train_features,train_target)
preds=model.predict(test_features)
eval_metric(test_target.values,preds,"SMAPE")

[37.65141857448004]